In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### Concatenate and Reshape
Here we load the data and then combine the two bands and recombine them into a single image/tensor for training

In [2]:
def load_and_format(in_path):
    out_df = pd.read_json(in_path)
    out_images = out_df.apply(lambda c_row: [np.stack([c_row['band_1'],c_row['band_2']], -1).reshape((75,75,2))],1)
    out_images = np.stack(out_images).squeeze()
    return out_df, out_images


In [3]:
base_path = 'data/iceberg'

In [4]:
train_df, train_images = load_and_format(os.path.join(base_path, 'train.json'))
print('training', train_df.shape, 'loaded', train_images.shape)
test_df, test_images = load_and_format(os.path.join(base_path, 'test.json'))
print('testing', test_df.shape, 'loaded', test_images.shape)
train_df.sample(3)

training (1604, 5) loaded (1604, 75, 75, 2)
testing (8424, 4) loaded (8424, 75, 75, 2)


,band_1,band_2,id,inc_angle,is_iceberg
1441,"[-31.115002, -32.274887, -34.369637, -36.11268...","[-29.176804, -31.115047, -31.115091, -28.75314...",f3dc5422,42.518,0
939,"[-24.579832, -21.058052, -19.792459, -21.58472...","[-31.127012, -30.600475, -29.1889, -29.188946,...",f29ff564,42.5222,0
285,"[-15.342882, -19.038483, -23.608122, -17.25976...","[-26.802034, -28.580906, -28.194857, -25.60223...",4b5201f0,39.6535,1


##### Simple CNN

In [5]:
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
X_train, X_test, y_train, y_test = train_test_split(train_images,
                                                   to_categorical(train_df['is_iceberg']),
                                                    random_state = 2017,
                                                    test_size = 0.5
                                                   )
print('Train', X_train.shape, y_train.shape)
print('Validation', X_test.shape, y_test.shape)

Using TensorFlow backend.


Train (802, 75, 75, 2) (802, 2)
Validation (802, 75, 75, 2) (802, 2)


In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, Dropout, MaxPooling2D, GlobalMaxPooling2D, Dense
simple_cnn = Sequential()
simple_cnn.add(BatchNormalization(input_shape = (75, 75, 2)))
for i in range(4):
    simple_cnn.add(Conv2D(8*2**i, kernel_size = (3,3)))
    simple_cnn.add(MaxPooling2D((2,2)))
simple_cnn.add(GlobalMaxPooling2D())
simple_cnn.add(Dropout(0.5))
simple_cnn.add(Dense(8))
simple_cnn.add(Dense(2, activation = 'softmax'))
simple_cnn.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
simple_cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 75, 75, 2)         8         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 73, 73, 8)         152       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 16)        1168      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 32)        4640      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 32)          0         
__________

In [7]:
simple_cnn.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, shuffle = True)

Train on 802 samples, validate on 802 samples
Epoch 1/10
802/802 [==============================] - 37s - loss: 0.6945 - acc: 0.5810 - val_loss: 1.4264 - val_acc: 0.5798
Epoch 2/10
802/802 [==============================] - 0s - loss: 0.5861 - acc: 0.6708 - val_loss: 0.5075 - val_acc: 0.7805
Epoch 3/10
802/802 [==============================] - 0s - loss: 0.5345 - acc: 0.7307 - val_loss: 0.4851 - val_acc: 0.6995
Epoch 4/10
802/802 [==============================] - 0s - loss: 0.4534 - acc: 0.7631 - val_loss: 0.4025 - val_acc: 0.8142
Epoch 5/10
802/802 [==============================] - 0s - loss: 0.3878 - acc: 0.8167 - val_loss: 0.5080 - val_acc: 0.7544
Epoch 6/10
802/802 [==============================] - 0s - loss: 0.4074 - acc: 0.8067 - val_loss: 0.3726 - val_acc: 0.8354
Epoch 7/10
802/802 [==============================] - 0s - loss: 0.3727 - acc: 0.8267 - val_loss: 0.3989 - val_acc: 0.7930
Epoch 8/10
802/802 [==============================] - 0s - loss: 0.3350 - acc: 0.8404 - val_

#### Make Predictions
Here we make predictions on the output and export the CSV so we can submit

In [8]:
test_predictions = simple_cnn.predict(test_images)

In [9]:
pred_df = test_df[['id']].copy()
pred_df['is_iceberg'] = test_predictions[:,1]
pred_df.to_csv('predictions.csv', index = False)
pred_df.sample(3)

,id,is_iceberg
7463,e3588c75,0.441332
4592,4caff066,0.017132
4119,753f66cc,0.010311
